In [8]:
import parselmouth
import time
import numpy as np
import speech_recognition as sr
import subprocess
from pythonosc.udp_client import SimpleUDPClient
from pythonosc import udp_client
from pythonosc import osc_bundle_builder
from pythonosc import osc_message_builder
import sounddevice as sd
import os
from flask_cors import CORS
from datetime import datetime
from flask import Flask, request, jsonify, render_template
import json
import threading
import cProfile
import atexit



In [46]:
# ADD:
# MESSAGE FOR CHANGING PITCH (done)
# AUDIO FILE PROCESSING SUPPORT + saving to audio file (done)
# INPUT/OUTPUT TIMESTAMP
# look at sampling size, make sure it is calculating correctly (yes, it's getting f1)
# have the hapticizer the same volume as speaker : NEED TO MAKE TIMEFRAME OF SOUND A LITTLE LONGER!!! (done)


# Create UDP client to send pitch to chuck code
client = SimpleUDPClient("127.0.0.1", 6449)

# Global variables - to store the audio parameters
hmin = 100
hmax = 300
vmin = 80
vmax = 400
fs = 44100
config = ""

chuck_process = None  # Global variable to store the Chuck process
chuck_lock = threading.Lock()

# Global variables - to store the modulation parameters
intensityFactor = 1.5
pitchFactor = 1.0
time_window = 0
duration = 0
waveform_shape = ""
sendPitch = True
sendIntensity = True

audio_thread_running = True


try:
    with open('config.json', 'r') as f:
        config = json.load(f)
except FileNotFoundError:
    print("Error: config.json not found. Using default values.")
    config = {
        'chuck_script': './chuckScripts/hapticize.ck',
        'default_intensity': 50,
        'default_window': 250,  # milliseconds
        'udp_ip': '127.0.0.1', # IP to send UDP packets to
        'udp_port': 6449, # Port to send UDP packets to
        'audio_file': './audio/angry.wav'
    }

def fade_pitch(old_val, new_val, total_duration, steps=3):
    if old_val == 0 or new_val == 0:
        # For silence/unvoiced words, just hold the new value
        client.send_message('/pitch', new_val)
        time.sleep(total_duration)
        return

    step_duration = total_duration / steps
    for i in range(1, steps + 1):
        interp = old_val + (new_val - old_val) * (i / steps)
        client.send_message('/pitch', interp)
        time.sleep(step_duration)


def process_audio():
    try:
        file_path = config['audio_file']
        if not os.path.exists(file_path):
            return jsonify({'error': 'File not found'}), 400

        sound = parselmouth.Sound(file_path)
        pitch = sound.to_pitch()
        intensity = sound.to_intensity()

        pitch_values = pitch.selected_array['frequency']
        intensity_values = intensity.values[0]
        max_intensity = np.max(intensity_values)
        min_intensity = np.min(intensity_values)
        timestep = pitch.dt
        total_frames = len(pitch_values)

        # Convert ms window to frame count
        window_size_ms = config.get('default_window', 250)
        window_size_frames = int((window_size_ms / 1000) / timestep)
        step_size = window_size_frames  # non-overlapping windows

        last_sent = 0

        for i in range(0, total_frames, step_size):
            pitch_window = pitch_values[i:i+window_size_frames]
            intensity_window = intensity_values[i:i+window_size_frames]

            # Filter out 0-pitch (unvoiced frames)
            voiced = [p for p in pitch_window if 80 <= p <= 300]
            avg_pitch = np.mean(voiced) if voiced else 0

            avg_intensity = np.mean(intensity_window)

            # Send pitch
            if avg_pitch > 80 and avg_pitch < 300:
                normalized_pitch = 100 + ((avg_pitch - 80) / (300 - 80)) * 200
                normalized_pitch = avg_pitch
                fade_pitch(last_sent, avg_pitch, window_size_ms / 1000)
                last_sent = avg_pitch
                #client.send_message('/pitch', normalized_pitch)

            # Normalize and send intensity
            if avg_intensity > 30:
                normalized_intensity = (avg_intensity - min_intensity) / (max_intensity - min_intensity)
                client.send_message('/loudness', normalized_intensity)

            time.sleep(window_size_ms / 1000)

        print("done")

    except Exception as e:
        print("hello", str(e))


# def process_audio():
#     try:
#         file_path = config['audio_file']
#         if not os.path.exists(file_path):
#             return jsonify({'error': 'File not found'}), 400

#         sound = parselmouth.Sound(file_path)
#         pitch = sound.to_pitch()
#         timestep = pitch.dt
#         intensity = sound.to_intensity()
#         print("here", pitch.dt)
#         # change the length of the frame to match window size

#         for frame in range(len(pitch.selected_array['frequency'])):
#             cur_pitch = pitch.selected_array['frequency'][frame]
#             cur_intensity = intensity.values[0][frame] if frame < len(intensity.values[0]) else 0

#             if 80 <= cur_pitch <= 300:
#                 normalized_pitch = 100 + ((cur_pitch - 80) / (300 - 80)) * 200
#                 client.send_message('/pitch', normalized_pitch)

#             if cur_intensity > 30:
#                 normalized_intensity = (cur_intensity - 30) / (100 - 30)
#                 client.send_message('/loudness', normalized_intensity)
            
#             time.sleep(timestep)

#         print("done")
#     except Exception as e:
#         print("hello", str(e))

process_audio()


done


In [21]:


# Function that starts chuck; currently causes a glitch in the chuck script
@app.route('/start_chuck', methods=['POST'])
def start_chuck():
    global chuck_process, hapticizing, audio_thread, chuck_lock
    hapticizing = True
    print("inside start")
    with chuck_lock:
        print("inside lock")
        if chuck_process is None:
            print("inside if")
            try:
                if not os.path.exists(config['chuck_script']):
                    print(f"Chuck script not found: {config['chuck_script']}")
                print("hello")
                chuck_process = subprocess.Popen(['chuck', config['chuck_script']],
                                                stdout=subprocess.PIPE,
                                                stderr=subprocess.PIPE,
                                                text=True)
                print("STARTED", chuck_process)

                # Start a thread to read the output of the Chuck process
                chuck_output_thread = threading.Thread(target=read_chuck_output)
                chuck_output_thread.daemon = True
                chuck_output_thread.start()

                return jsonify({'status': 'Hapticizing started'})
            except Exception as e:
                print(f"Error starting Chuck: {e}")
                return jsonify({'error': str(e)}), 500
        else:
            print("um")

# Debugging function to read chuck output
def read_chuck_output():
    global chuck_process
    while chuck_process and chuck_process.poll() is None:
        output = chuck_process.stdout.readline()
        if output:
            print(output.strip())

# Function that stops chuck
def stop_chuck():
    global chuck_process, chuck_lock
    with chuck_lock:
        if chuck_process:
            chuck_process.terminate()  # Or .kill() if needed
            chuck_process.wait() # Make sure the process has fully stopped
            chuck_process = None
            print("Chuck stopped.")

# Starts the hapticize.ck script
@app.route('/start_hapticize', methods=['POST'])
def start_hapticize():
    global hapticizing, chuck_process, audio_thread
    # if chuck_process is None:
    #     #start_chuck()
    hapticizing = True
    print("inside start hapticize")
    #start_chuck()
    print("chuck process", chuck_process)
    if chuck_process != None:
        print("chuck started")
    else:
        print("CHUCK NOT STARTED")

    return jsonify({'status': 'Hapticizing started'})  # Just sets the flag

# Function that stops the hapticize.ck script
@app.route('/stop_hapticize', methods=['POST'])
def stop_hapticize():
    global hapticizing, chuck_process
    hapticizing = False
    stop_chuck()
    chuck_process = None
    return jsonify({'status': 'Hapticizing stopped'})

# Function that shuts down the server
@app.route('/shutdown')
def shutdown_server():
    func = request.environ.get('werkzeug.server.shutdown')
    if func is None:
        raise RuntimeError('Not running with the Werkzeug Server')
    func()
    return "Server shutting down..."

# Function that sets the modulation parameters based on what is received from the front end
@app.route('/modulation', methods=['POST'])
def modulation():
    global intensityFactor, pitchFactor, time_window, waveform_shape, sendPitch, sendIntensity
    try:
        data = request.get_json()  # Get the JSON data from the request body

        # Extract the modulation parameters from the JSON data
        intensityFactor = float(data.get('intensity'))
        pitchFactor = float(data.get('pitch'))
        time_window = float(data.get('timeWindow'))
        waveform_shape = data.get('waveformShape')
        sendPitch = data.get('sendPitch')
        sendIntensity = data.get('sendVolume')

        bundle = osc_bundle_builder.OscBundleBuilder(osc_bundle_builder.IMMEDIATELY)

        msg = osc_message_builder.OscMessageBuilder(address="/modulation/bundle")
        msg.add_arg(intensityFactor)
        msg.add_arg(pitchFactor)
        msg.add_arg(time_window)
        msg.add_arg(waveform_shape)
        bundle.add_content(msg.build())

        client.send(bundle.build())

        print("SENT BUNDLE")

        # Example: Print the received parameters to the console
        print("Received modulation parameters:")
        print(f"Intensity: {intensityFactor}")
        print(f"Pitch: {pitchFactor}")
        print(f"Time Window: {time_window}")
        print(f"Waveform Shape: {waveform_shape}")
        print(f"Send pitch: {sendPitch}")
        print(f"Send intensity: {sendIntensity}")

        # Example: Send OSC messages to Chuck (replace with your actual logic)
        # client.send_message("/intensity", intensity)  # Assuming /intensity is the correct address
        # client.send_message("/pitch", pitch)        # Assuming /pitch is the correct address
        # ... send other OSC messages

        return jsonify({'status': 'Modulation parameters received'}), 200  # Return a success response

    except Exception as e:
        print(f"Error processing modulation parameters: {e}")
        return jsonify({'error': str(e)}), 500

# IN PROGRESS: Function that will buzz one time to show what current configurations will do
@app.route('/vibrate', methods=['POST'])
def vibrate():
    try:
        data = request.get_json()
        intensity_exaggeration = float(data.get('intensityExaggeration', 1.0)) # Get modulation values
        pitch_exaggeration = float(data.get('pitchExaggeration', 1.0))
        time_window = int(data.get('timeWindow', 250))
        waveform_shape = data.get('waveformShape', 'sine')

        modintensity = 0.5 * intensity_exaggeration
        modpitch = 180 * pitch_exaggeration

        client.send_message("/pitch", modpitch)
        client.send_message("/intensity", modintensity)
        if "Error:" not in result: # If there's no error in the function
            return jsonify({'status': 'vibrating', 'intensity': intensity, 'pitch': pitch, 'form': waveform_shape})
        else:
            return jsonify({'status': 'error', 'message': result}), 500 # If there's an error

    except Exception as e:
        print(f"Error in /vibrate route: {e}")
        return jsonify({'status': 'error', 'message': str(e)}), 500

NameError: name 'app' is not defined

In [ ]:
# Function that detects pitch and intensity in real time and sends it to the chuck script
def detect_and_send_pitch(audio, sample_rate):
    global intensityFactor, pitchFactor
    try:
        # Convert audio to a Parselmouth Sound object
        sound = parselmouth.Sound(audio, sampling_frequency=sample_rate)
        rms = sound.get_rms()
        intensity = sound.get_intensity()
        #print(intensity)
            
        # Sound intensity threshold: if sound is less than 30 dB, ignore it
            # Probably needs to be higher than 30 in practice especially in a noisy environment
            # if intensity < 30:
            #     return
        #print("detect and send pitch")
        # Extract pitch using Parselmouth
        pitch = sound.to_pitch(pitch_floor=120)
        pitch_values = pitch.selected_array['frequency']

        total = 0.0
        valid = 0

        mode = -1

        ##IDEA: LOOK FOR MODE INSTEAD

        for value in pitch_values:
            # Detect pitches in human voice range: 80-300 Hz
            if value != 0:
                total = total + value
                valid += 1
        if valid > 0:
            value = total / valid
        else:
            value = 0

        vmin = 80
        vmax = 400
        
        if value > vmin and value < vmax and intensity > 30:
            # Normalize to haptic range: 100-300 Hz (SUBJECT TO CHANGE)
            hmin = 100
            hmax = 300
            normalized_pitch = hmin + ((value - vmin) / (vmax - vmin)) * hmax
            #print(sendPitch, pitchFactor)
            #if sendPitch:
            client.send_message("/pitch", normalized_pitch * pitchFactor)
                #print("PITCH", pitchFactor, sendPitch, "normal", normalized_pitch, "augmented", normalized_pitch * pitchFactor, datetime.now().strftime("%S"))

        if intensity > 30:
            normalized_intensity = (intensity - 30) / (100 - 30)
            #if sendIntensity:
            
            #if sendIntensity:
            intensityFactor = 1
            client.send_message("/loudness", normalized_intensity * intensityFactor)
            print("INTENSITY", "factor", intensityFactor, "number", normalized_intensity, "both", intensityFactor * normalized_intensity, datetime.now().strftime("%S"))
            #print("INTENSITY", intensityFactor, sendIntensity, "normal", normalized_intensity, "augmented", float(normalized_intensity) * intensityFactor, datetime.now().strftime("%S"))
            #print(threading.enumerate())
        
            
            
    except Exception as e:
        print(f"Error in detect_and_send_pitch: {e}")
        return f"Error: {e}"

def audio_callback(indata, frames, time, status):
    if status:
        print(status)
        return
    
    mono_audio = np.mean(indata, axis=1)
    detect_and_send_pitch(mono_audio, fs)

#THIS WINDOW IS WHAT CHANGES THE TIMEFRAME OF THE SOUND
def audio_thread_function():
    num = time_window / 1000
    window = int(fs * num)
    window = int(fs * 0.25)

    with sd.InputStream(channels=1, samplerate=fs, blocksize=window, callback=audio_callback) as stream:
        print("Listening (audio thread)...")
        while audio_thread_running:
            time.sleep(0.1)

def stop_audio_thread():
    global audio_thread_running
    audio_thread_running = False
    print("Audio thread stopped.")

print("Welcome to Hapticizer 3000. Enter a number:")
choice = input("(1) Hapticize an audio file \n(2) Hapticize real-time audio\n")

audio_thread = threading.Thread(target=audio_thread_function)
audio_thread.daemon = True
audio_thread.start()

# def monitor_threads():
#     while True:
#         print("Active threads:")
#         for thread in threading.enumerate():
#             print(f"Thread {thread.name}: {thread.is_alive()}")
#         time.sleep(5)

# thread_monitor = threading.Thread(target=monitor_threads)
# thread_monitor.daemon = True
# thread_monitor.start()

atexit.register(stop_audio_thread)

app.run(debug=True, use_reloader=False)
print(threading.enumerate())
print("stream closed")
if chuck_process != None:
    stop_chuck()





 * Serving Flask app '__main__'
 * Debug mode: on


Listening (audio thread)...


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


INTENSITY factor 1 number 0.2100325575905907 both 0.2100325575905907 22
INTENSITY factor 1 number 0.2878813043218925 both 0.2878813043218925 22
INTENSITY factor 1 number 0.5147068757639456 both 0.5147068757639456 22
INTENSITY factor 1 number 0.30658391243573674 both 0.30658391243573674 22
INTENSITY factor 1 number 0.4345100655639727 both 0.4345100655639727 28
INTENSITY factor 1 number 0.5598630359863157 both 0.5598630359863157 28
INTENSITY factor 1 number 0.3879046171836617 both 0.3879046171836617 28
INTENSITY factor 1 number 0.03307337175021437 both 0.03307337175021437 29
INTENSITY factor 1 number 0.2181815981528266 both 0.2181815981528266 29
INTENSITY factor 1 number 0.09732239066249783 both 0.09732239066249783 30
INTENSITY factor 1 number 0.10795223665366355 both 0.10795223665366355 30
INTENSITY factor 1 number 0.0868816424534118 both 0.0868816424534118 30
INTENSITY factor 1 number 0.10676371087280997 both 0.10676371087280997 31
INTENSITY factor 1 number 0.17288400626222145 both 0.1

INTENSITY factor 1 number 0.3369352837604014 both 0.3369352837604014 20
INTENSITY factor 1 number 0.5820426528889961 both 0.5820426528889961 20
INTENSITY factor 1 number 0.30292302308069735 both 0.30292302308069735 20
INTENSITY factor 1 number 0.15882562201775172 both 0.15882562201775172 21
INTENSITY factor 1 number 0.13152958971848575 both 0.13152958971848575 21
INTENSITY factor 1 number 0.019807002092495654 both 0.019807002092495654 22
INTENSITY factor 1 number 0.25292302247694626 both 0.25292302247694626 27
INTENSITY factor 1 number 0.15436170448717487 both 0.15436170448717487 28
INTENSITY factor 1 number 0.002247197722343941 both 0.002247197722343941 28
INTENSITY factor 1 number 0.20899161670014074 both 0.20899161670014074 29
INTENSITY factor 1 number 0.18263507101246393 both 0.18263507101246393 29
INTENSITY factor 1 number 0.12980561816637706 both 0.12980561816637706 29
INTENSITY factor 1 number 0.18489133323176696 both 0.18489133323176696 29
INTENSITY factor 1 number 0.2144413834

In [ ]:
%tb

SystemExit: 1

In [ ]:


print("Welcome to Hapticizer 3000. Enter a number:")
choice = input("(1) Hapticize an audio file \n(2) Hapticize real-time audio\n")

if choice == "1":
    filename = input("What file would you like to hapticize? ")
    sound = parselmouth.Sound(filename)
    pitch = sound.to_pitch()
    intensity = sound.to_intensity()
    intensity_values = intensity.values[0]
    timestep = pitch.dt
    #print("FRAME LENGTH: ", pitch.dt)
    #print("length", len(pitch.selected_array["frequency"]))
    input("Start the hapticizer and then press enter to start.")
    #Frame length is 0.01 sec
    #To make chunk size the same as real-time, we need to get avg pitch of every 50 samples
    pitch_total = 0.0
    intensity_total = 0.0
    for frame in range(len(pitch.selected_array["frequency"])):
        curpitch = pitch.selected_array["frequency"][frame]
        curintensity = 0.0
        if frame < len(intensity_values):
            curintensity = intensity_values[frame]
        #else:
            #print("outside intensity frame?")
        if frame % 25 != 0:
            pitch_total += curpitch
            intensity_total += curintensity
        else:
            curpitch = pitch_total / 25
            curintensity = intensity_total / 25
            # if frame < len(intensity_values):
            #     curintensity = intensity_values[frame]
            #print("curpitch", curpitch, "curintensity", curintensity)
            if curpitch > 80 and curpitch < 1000:
                        
                normalized_pitch = hmin + ((curpitch - vmin) / (vmax - vmin)) * hmax
                                # Normalize intensity to Chuck's gain range, 0-1
                                # Guesstimating intensity range to be like 30-100 dB
                
                #print("Sending pitch", normalized_pitch)
                client.send_message("/pitch", float(normalized_pitch))
                

            if curintensity > 20:
                normalized_intensity = (curintensity - 30) / (100 - 30)
                #print("Sending loudness", normalized_intensity)
                client.send_message("/loudness", normalized_intensity)
                # if curintensity < 50:
                #     client.send_message("/loudness", 0.1)
                # elif curintensity > 70:
                #     client.send_message("/loudness", 1.0)
                # else:
                #     client.send_message("/loudness", 0.5)
            
            pitch_total = 0.0
            intensity_total = 0.0
            
        time.sleep(timestep)

    print("Done!")

else:        

    def detect_and_send_pitch(audio, sample_rate):
        # Convert audio to a Parselmouth Sound object
        sound = parselmouth.Sound(audio, sampling_frequency=sample_rate)
        rms = sound.get_rms()
        intensity = sound.get_intensity()
        #print("Intensity", sound.get_intensity())

        # Sound intensity threshold: if sound is less than 30 dB, ignore it
        # Probably needs to be higher than 30 in practice especially in a noisy environment
        # if intensity < 30:
        #     return
        #print("detect and send pitch")
        # Extract pitch using Parselmouth
        pitch = sound.to_pitch(pitch_floor=120)
        #print("FRAME LENGTH", pitch.dt)
        #print("whole thing length", pitch.get_total_duration())
        pitch_values = pitch.selected_array['frequency']

        total = 0.0

        for value in pitch_values:
            # Detect pitches in human voice range: 80-300 Hz
            total = total + value
        
        value = total / len(pitch_values)
        #print("VALUE", value)

        vmin = 80
        vmax = 400
        if value > vmin and value < vmax:
            #print(value)

            # Normalize to haptic range: 100-300 Hz (SUBJECT TO CHANGE)
            hmin = 100
            hmax = 300
            normalized_pitch = hmin + ((value - vmin) / (vmax - vmin)) * hmax
            client.send_message("/pitch", normalized_pitch)
            # Normalize intensity to Chuck's gain range, 0-1
            # Guesstimating intensity range to be like 30-100 dB

        if intensity > 20:
            normalized_intensity = (intensity - 30) / (100 - 30)
            client.send_message("/loudness", normalized_intensity)
            # if intensity < 50:
            #     client.send_message("/loudness", 0.1)
            # elif intensity > 70:
            #     client.send_message("/loudness", 1.0)
            # else:
            #     client.send_message("/loudness", 0.5)
            #time.sleep(0.2)

        
        

    def audio_callback(indata, frames, time, status):
        #print("callback")
        if status:
            print(status)
        mono_audio=np.mean(indata, axis=1)
        detect_and_send_pitch(mono_audio, fs)

    #Duration will be the length of the chunk used to get the pitch and intensity
    #Currently 0.5 seconds
    duration = int(fs * 0.25)

    with sd.InputStream(channels=1, samplerate=fs, blocksize=duration, callback=audio_callback):
        print("Listening...")
        while True:
            pass



    # General function for analyzing sound
    def analyze_sound(sound):
        analysis = {"pitch":[], "intensity":[], "avgPitch":0, "avgIntensity":0, "times":[]}
        pitch = sound.to_pitch()
        analysis["pitch"] = pitch.values[0]
        analysis["avgPitch"] = pitch.selected_array['frequency']

        intensity = sound.to_intensity()
        analysis["times"] = intensity.xs()
        analysis["intensity"] = intensity.values[0]
        analysis["avgIntensity"] = intensity.get_average()
        print(analysis["avgIntensity"])
        return analysis

    # Previous attempt at sending sound (slow and finicky)

    def send_intensity_to_chuck(chuck_instance, times, intensity_values, output_wav):
        
        new_intensity = []

        #python library that detects voice; when voice off, play the vibration for the previous utterance (maximum time frame?)
        
        #approach that records some and then processes it with delay
        r = sr.Recognizer()
        r.pause_threshold = 0.8 # this is the default; can be changed
        with sr.Microphone() as source:
            audio = r.listen(source)
            wav_data = audio.get_wav_data()
            cur_sound = parselmouth.Sound(wav_data, audio.sample_rate)
            analysis = analyze_sound(cur_sound)


        # can it be even more real time.... ponder 
        for time_step, intensity in zip(analysis["times"], analysis["intensity"]):
            #normalize intensity and pitch
            s.setGain(intensity)

        subprocess.run(["chuck", "hapticize.ck"])
        chuck_instance.run("""
        SinOsc s => Gain g => dac; // Sine oscillator
        s => WvOut w => blackhole;

        "{output}" => w.wavFilename;

        200 => s.freq;   // Base frequency
        0.1::second => dur d; // Time step

        fun void updateGain(float newGain) {
            newGain => s.gain; // Update the gain based on intensity
        }

        while (true) {
            1::second => now; // Keep the ChucK VM running
        }
        """)

        # Send the intensity data to ChucK
        for time_step, intensity in zip(times, intensity_values):
            chuck_instance.call("updateGain", [float(intensity)])
            time.sleep(time_step)

    # output = open("output.wav", "wb")
    # send_intensity_to_chuck(chuck_instance, times, values, output)

    #if [acoustic property] > number, add a haptic vibration for it

Welcome to Hapticizer 3000. Enter a number:
INTENSITY factor 1 number 0.1402906133803114 both 0.1402906133803114 03
INTENSITY factor 1 number 0.19718741010817006 both 0.19718741010817006 03
INTENSITY factor 1 number 0.11029596153825645 both 0.11029596153825645 04
INTENSITY factor 1 number 0.2562311067638291 both 0.2562311067638291 04
INTENSITY factor 1 number 0.10844546400841862 both 0.10844546400841862 04
Done!


INTENSITY factor 1 number 0.010227286585886475 both 0.010227286585886475 06
INTENSITY factor 1 number 0.19213277061622916 both 0.19213277061622916 08
INTENSITY factor 1 number 0.09833875593703649 both 0.09833875593703649 08
INTENSITY factor 1 number 0.09237356949323987 both 0.09237356949323987 10
INTENSITY factor 1 number 0.017430421975048408 both 0.017430421975048408 10
INTENSITY factor 1 number 0.22309015575421115 both 0.22309015575421115 12
INTENSITY factor 1 number 0.09664303649562771 both 0.09664303649562771 12
INTENSITY factor 1 number 0.0028931360326995203 both 0.0028931360326995203 27
INTENSITY factor 1 number 0.025608260704892163 both 0.025608260704892163 27
INTENSITY factor 1 number 0.34648603976975617 both 0.34648603976975617 28
INTENSITY factor 1 number 0.5012947834131696 both 0.5012947834131696 28
INTENSITY factor 1 number 0.2070186683031229 both 0.2070186683031229 28
INTENSITY factor 1 number 0.621766221288752 both 0.621766221288752 29
INTENSITY factor 1 number 0.71780027